# Предобработка текстовых данных

Работа с текстовыми данными обычно начинается с предобработки. Грубо говоря, предобработка - это удаление всего лишнего и приведение к нужному формату. Предобработка может быть долгой и неприятной, но в большинстве случаев все сводится к использованию стандартных инструментов. Эта тетрадка познакомит вас с такими инструментами.

#### Установите все нужные библиотеки. Подробнее про каждую из них ниже

In [1]:
!pip install pymystem3
!pip install pymorphy2[fast]
!pip install razdel
!pip install gensim
!pip install nltk
!pip install rusenttokenize
!pip install regex

In [125]:
# Скорее всего вы уже знакомы с библиотекой pymorphy2.
# pymorphy2[fast] - это оптимизированный pymorphy2, который работает точно также, но сильно быстрее
# Если у вас windows, то он вряд ли установится и вам придется пользоваться стандартным

### Регулярные выражения

Один из базовых, но в то же время самых полезных инструментов для предобработки текста - регулярные выражения. В вводной части курса им был посвящен целый семинар - https://github.com/mannefedov/compling_nlp_hse_course/blob/master/notebooks/first_module_intro/01_regular_expressions.ipynb  

Если вы не ходили подготовительную часть и чувствуете себя неуверенно при работе с регулярками - пройдитесь по семинару. Решить домашку тоже не помешает. В семинаре я буду предполагать, что вы уже знакомы с вещами, которые разобраны в подготовительной части. Но новые вещи я буду объяснять дополнительно!

## Разбиение на предложения, токенизация, нормализация

За каждым из этих трех терминов стоит большая и сложная подзадача NLP. Однако для каждой есть готовые решения, которые очень хорошо работают. 

### Разбиение на предложения

In [41]:
text = """Задача NLI важна для компьютерных лингвистов, ибо она позволяет детально рассмотреть, какие языковые явления данная модель понимает хорошо, а на каких – "плывёт"; по этому принципу устроены диагностические датасеты SuperGLUE и RussianSuperGLUE. Кроме этого, модели NLI обладают прикладной ценностью по нескольким причинам.

Во-первых, NLI можно использовать для контроля качества генеративных моделей. Есть масса задач, где на основе текста X нужно сгенерировать близкий к нему по смыслу текст Y: суммаризация, упрощение текстов, перефразирование, перенос стиля на текстах, текстовые вопросно-ответные системы, и даже машинный перевод. Современные seq2seq нейросети типа T5 (которая в этом году появилась и для русского языка) в целом неплохо справляются с такими задачами, но время от времени лажают, упуская какую-то важную информацию из Х, или, наоборот, дописывая в текст Y что-то нафантазированное "от себя". С помощью модели NLI можно проверять, что из X следует Y (то есть в новом тексте нету "отсебятины", придуманной моделью), и что из Y следует X (т.е. вся информация, присутствовавшая в исходном тексте, в новом также отражена).

Во-вторых, с помощью моделей NLI можно находить нетривиальные парафразы и в целом определять смысловую близость текстов. Для русского языка уже существует ряд моделей и датасетов по перефразированию, но кажется, что можно сделать ещё больше и лучше. В статье Improving Paraphrase Detection with the Adversarial Paraphrasing Task предложили считать парафразами такую пару предложений, в которой каждое логически следует из другого – и это весьма логично. Поэтому модели NLI можно использовать и для сбора обучающего корпуса парафраз (и не-парафраз, если стоит задача их детекции), и для фильтрации моделей, генерирующих парафразы.

"""
# текст отсюда - https://habr.com/ru/post/582620/

В __nltk__ есть уже готовая функция для разбивки на предложения. 

In [116]:
from nltk import sent_tokenize

In [117]:
sent_tokenize(text, 'russian')[:10]

['Задача NLI важна для компьютерных лингвистов, ибо она позволяет детально рассмотреть, какие языковые явления данная модель понимает хорошо, а на каких – "плывёт"; по этому принципу устроены диагностические датасеты SuperGLUE и RussianSuperGLUE.',
 'Кроме этого, модели NLI обладают прикладной ценностью по нескольким причинам.',
 'Во-первых, NLI можно использовать для контроля качества генеративных моделей.',
 'Есть масса задач, где на основе текста X нужно сгенерировать близкий к нему по смыслу текст Y: суммаризация, упрощение текстов, перефразирование, перенос стиля на текстах, текстовые вопросно-ответные системы, и даже машинный перевод.',
 'Современные seq2seq нейросети типа T5 (которая в этом году появилась и для русского языка) в целом неплохо справляются с такими задачами, но время от времени лажают, упуская какую-то важную информацию из Х, или, наоборот, дописывая в текст Y что-то нафантазированное "от себя".',
 'С помощью модели NLI можно проверять, что из X следует Y (то есть

Nltk также позволяет обучить свой токенизатор предложений под определенный корпус.
Вот код для примера (изначально я взял его отсюда - https://nlpforhackers.io/splitting-text-into-sentences/ , но сайт больше не работает):



In [329]:
from pprint import pprint
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer
 
trainer = PunktTrainer()
trainer.INCLUDE_ALL_COLLOCS = True
trainer.train(text)
 
tokenizer = PunktSentenceTokenizer(trainer.get_params())
 
# Test the tokenizer on a piece of text
sentences = "Mr. James told me Dr. Brown is not available today. I will try tomorrow."
 
print(tokenizer.tokenize(sentences))
# ['Mr. James told me Dr.', 'Brown is not available today.', 'I will try tomorrow.']
 
# View the learned abbreviations
print(tokenizer._params.abbrev_types)
# set([...])
 
# Here's how to debug every split decision
for decision in tokenizer.debug_decisions(sentences):
    pprint(decision)
    print('=' * 30)

# adding abbreviations manually
tokenizer._params.abbrev_types.add('dr')
 
print(tokenizer.tokenize(sentences))
# ['Mr. James told me Dr. Brown is not available today.', 'I will try tomorrow.']
 
for decision in tokenizer.debug_decisions(sentences):
    pprint(decision)
    print('=' * 30)

В __gensim__ тоже есть готовая функция

In [119]:
from gensim.summarization.textcleaner import split_sentences

In [120]:
# это ещё и генератор, т.е. сразу подходит для больших корпусов
list(split_sentences(text))[:5]

['Задача NLI важна для компьютерных лингвистов, ибо она позволяет детально рассмотреть, какие языковые явления данная модель понимает хорошо, а на каких – "плывёт"; по этому принципу устроены диагностические датасеты SuperGLUE и RussianSuperGLUE.',
 'Кроме этого, модели NLI обладают прикладной ценностью по нескольким причинам.',
 'Во-первых, NLI можно использовать для контроля качества генеративных моделей.',
 'Есть масса задач, где на основе текста X нужно сгенерировать близкий к нему по смыслу текст Y: суммаризация, упрощение текстов, перефразирование, перенос стиля на текстах, текстовые вопросно-ответные системы, и даже машинный перевод.',
 'Современные seq2seq нейросети типа T5 (которая в этом году появилась и для русского языка) в целом неплохо справляются с такими задачами, но время от времени лажают, упуская какую-то важную информацию из Х, или, наоборот, дописывая в текст Y что-то нафантазированное "от себя".']

У DeepPavlov есть библиотека [**rusenttokenizer**](https://github.com/deepmipt/ru_sentence_tokenizer).

In [121]:
from rusenttokenize import ru_sent_tokenize

In [122]:
ru_sent_tokenize(text)[:10]

['Задача NLI важна для компьютерных лингвистов, ибо она позволяет детально рассмотреть, какие языковые явления данная модель понимает хорошо, а на каких – "плывёт"; по этому принципу устроены диагностические датасеты SuperGLUE и RussianSuperGLUE.',
 'Кроме этого, модели NLI обладают прикладной ценностью по нескольким причинам.',
 'Во-первых, NLI можно использовать для контроля качества генеративных моделей.',
 'Есть масса задач, где на основе текста X нужно сгенерировать близкий к нему по смыслу текст Y: суммаризация, упрощение текстов, перефразирование, перенос стиля на текстах, текстовые вопросно-ответные системы, и даже машинный перевод.',
 'Современные seq2seq нейросети типа T5 (которая в этом году появилась и для русского языка) в целом неплохо справляются с такими задачами, но время от времени лажают, упуская какую-то важную информацию из Х, или, наоборот, дописывая в текст Y что-то нафантазированное "от себя".',
 'С помощью модели NLI можно проверять, что из X следует Y (то есть

В проекте Natasha есть библиотека [**razdel**](https://github.com/natasha/razdel). Она чуть более навороченная.

In [123]:
from razdel import sentenize

In [124]:
sents = list(sentenize(text))

In [126]:
# числа тут - это спаны, индексы начала и конца предложения в изначальном тексте
sents[:5]

[Substring(0,
           244,
           'Задача NLI важна для компьютерных лингвистов, ибо она позволяет детально рассмотреть, какие языковые явления данная модель понимает хорошо, а на каких – "плывёт"; по этому принципу устроены диагностические датасеты SuperGLUE и RussianSuperGLUE.'),
 Substring(245,
           322,
           'Кроме этого, модели NLI обладают прикладной ценностью по нескольким причинам.'),
 Substring(324,
           401,
           'Во-первых, NLI можно использовать для контроля качества генеративных моделей.'),
 Substring(402,
           635,
           'Есть масса задач, где на основе текста X нужно сгенерировать близкий к нему по смыслу текст Y: суммаризация, упрощение текстов, перефразирование, перенос стиля на текстах, текстовые вопросно-ответные системы, и даже машинный перевод.'),
 Substring(636,
           913,
           'Современные seq2seq нейросети типа T5 (которая в этом году появилась и для русского языка) в целом неплохо справляются с такими задачам

In [127]:
# у объекта Substring есть атрибуты start, stop и text. С помощью них можно вытащить нужное
[sent.text for sent in sents[:5]]

['Задача NLI важна для компьютерных лингвистов, ибо она позволяет детально рассмотреть, какие языковые явления данная модель понимает хорошо, а на каких – "плывёт"; по этому принципу устроены диагностические датасеты SuperGLUE и RussianSuperGLUE.',
 'Кроме этого, модели NLI обладают прикладной ценностью по нескольким причинам.',
 'Во-первых, NLI можно использовать для контроля качества генеративных моделей.',
 'Есть масса задач, где на основе текста X нужно сгенерировать близкий к нему по смыслу текст Y: суммаризация, упрощение текстов, перефразирование, перенос стиля на текстах, текстовые вопросно-ответные системы, и даже машинный перевод.',
 'Современные seq2seq нейросети типа T5 (которая в этом году появилась и для русского языка) в целом неплохо справляются с такими задачами, но время от времени лажают, упуская какую-то важную информацию из Х, или, наоборот, дописывая в текст Y что-то нафантазированное "от себя".']

Если все-таки нужно добавить каких-то специфичных правил разбиения на предложения, можно опять же воспользоваться регулярными выражениями. Однако в этом случае регулярка будет посложнее. 

Давайте посмотрим, что получится, если в качестве разделителя использовать !?. пробел и заглавную букву.  

In [128]:
re.split('[!?\.] [А-Я]', text)[:10]

['Задача NLI важна для компьютерных лингвистов, ибо она позволяет детально рассмотреть, какие языковые явления данная модель понимает хорошо, а на каких – "плывёт"; по этому принципу устроены диагностические датасеты SuperGLUE и RussianSuperGLUE',
 'роме этого, модели NLI обладают прикладной ценностью по нескольким причинам.\n\nВо-первых, NLI можно использовать для контроля качества генеративных моделей',
 'сть масса задач, где на основе текста X нужно сгенерировать близкий к нему по смыслу текст Y: суммаризация, упрощение текстов, перефразирование, перенос стиля на текстах, текстовые вопросно-ответные системы, и даже машинный перевод',
 'овременные seq2seq нейросети типа T5 (которая в этом году появилась и для русского языка) в целом неплохо справляются с такими задачами, но время от времени лажают, упуская какую-то важную информацию из Х, или, наоборот, дописывая в текст Y что-то нафантазированное "от себя"',
 ' помощью модели NLI можно проверять, что из X следует Y (то есть в новом 

Проблема в том, что сам разделитель удаляется тоже, а нам нужно удалить только пробел между знаками препинания и заглавной буквой.

Решается эта проблема с помощью __look ahead__ и __look behind__ (название функционала в регулярных выражениях).  
Синтаксис там такой:  
 **(?<=pattern)** положительное look-behind условие  
 **(?<!pattern)** отрицательное look-behind условие   
 **(?=pattern)** положительное look-ahead условие   
 **(?!pattern)** отрицательное look-ahead условие   

Подробно про это написано тут: https://www.regular-expressions.info/lookaround.html  

Look behind и look ahead превращают паттерн в условный, то есть проверяется есть ли он (до или после, соответственно), но его захвата не происходит. 

Обернём наше регулярное выражение и посмотрим, что получается:

In [129]:
re.split('(?<=[\.?!]) +(?=[А-ЯЁ])', text.replace('\n', ' '))[:10]

['Задача NLI важна для компьютерных лингвистов, ибо она позволяет детально рассмотреть, какие языковые явления данная модель понимает хорошо, а на каких – "плывёт"; по этому принципу устроены диагностические датасеты SuperGLUE и RussianSuperGLUE.',
 'Кроме этого, модели NLI обладают прикладной ценностью по нескольким причинам.',
 'Во-первых, NLI можно использовать для контроля качества генеративных моделей.',
 'Есть масса задач, где на основе текста X нужно сгенерировать близкий к нему по смыслу текст Y: суммаризация, упрощение текстов, перефразирование, перенос стиля на текстах, текстовые вопросно-ответные системы, и даже машинный перевод.',
 'Современные seq2seq нейросети типа T5 (которая в этом году появилась и для русского языка) в целом неплохо справляются с такими задачами, но время от времени лажают, упуская какую-то важную информацию из Х, или, наоборот, дописывая в текст Y что-то нафантазированное "от себя".',
 'С помощью модели NLI можно проверять, что из X следует Y (то есть

## Токенизация

Мы разбили текст на предложения. Теперь предложения нужно разбить на токены. Под токенами обычно понимаются слова, но это могут быть и какие-то более длинные или короткие куски. 

Самый простой способ токенизации -- стандартный питоновский __str.split__ метод.  
По умолчанию он разбивает текст по последовательностям пробелов 

In [130]:
'1  2 3'.split(' ') # NB! .split() и .split(' ') - не одно и тоже

['1', '', '2', '3']

In [131]:
'1  2 3'.split()

['1', '2', '3']

In [134]:
text.split()[10:30]

['рассмотреть,',
 'какие',
 'языковые',
 'явления',
 'данная',
 'модель',
 'понимает',
 'хорошо,',
 'а',
 'на',
 'каких',
 '–',
 '"плывёт";',
 'по',
 'этому',
 'принципу',
 'устроены',
 'диагностические',
 'датасеты',
 'SuperGLUE']

Большая часть слов отделяется, но знаки препинания лепятся к словам.
Можно пройтись по всем словам и убрать из них пунктуацию с методом str.strip.

In [99]:
#основные знаки преминания хранятся в питоновском модуле string в punctuation
import string

In [139]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [140]:
# в этом списке не хватает кавычек-ёлочек, лапок, длинного тире и многоточия
string.punctuation += '«»—…“”'

In [142]:
[word.strip(string.punctuation) for word in text.split()][10:30]

['рассмотреть',
 'какие',
 'языковые',
 'явления',
 'данная',
 'модель',
 'понимает',
 'хорошо',
 'а',
 'на',
 'каких',
 '–',
 'плывёт',
 'по',
 'этому',
 'принципу',
 'устроены',
 'диагностические',
 'датасеты',
 'SuperGLUE']

Так не будут удаляться дефисы и точки в сокращениях, не разделенных пробелом.

In [146]:
'как-нибудь'.strip(string.punctuation)

'как-нибудь'

In [145]:
'т.е.'.strip(string.punctuation)

'т.е'

Такой способ токенизации может работать быстрее других из-за того, что используются только дефолтные инструменты питона. Если важно качество, то лучше пользоваться готовыми токенизаторами

Например, готовые токенизаторы есть в nltk. Они не удаляют пунктуацию, а выделяют её отдельным токеном.

**wordpunct_tokenizer** разбирает по регулярке - *'\w+|[^\w\s]+'* (попробуйте понять как она работает просто глядя на паттерн)

In [149]:
from nltk.tokenize import word_tokenize, wordpunct_tokenize

In [150]:
wordpunct_tokenize(text)[:10]

['Задача',
 'NLI',
 'важна',
 'для',
 'компьютерных',
 'лингвистов',
 ',',
 'ибо',
 'она',
 'позволяет']

**word_tokenize** также построен на регулярках, но они там более сложные (учитывается последовательность некоторых 
символов, символы начала, конца слова и т.д). 

Специально подобранного под русский язык токенизатора там нет, 
но и с английским всё работает достаточно хорошо --
сокращения типа т.к собираются в один токен, дефисные слова тоже не разделяются, многоточия тут тоже не отделяются, но это можно поправить.

In [158]:
word_tokenize(text)[130:150]

['лажают',
 ',',
 'упуская',
 'какую-то',
 'важную',
 'информацию',
 'из',
 'Х',
 ',',
 'или',
 ',',
 'наоборот',
 ',',
 'дописывая',
 'в',
 'текст',
 'Y',
 'что-то',
 'нафантазированное',
 '``']

В генсиме тоже есть функция для токенизации

In [153]:
from gensim.utils import tokenize

In [155]:
# опять же, это генератор
list(tokenize(text, lowercase=True))[30:50]

['russiansuperglue',
 'кроме',
 'этого',
 'модели',
 'nli',
 'обладают',
 'прикладной',
 'ценностью',
 'по',
 'нескольким',
 'причинам',
 'во',
 'первых',
 'nli',
 'можно',
 'использовать',
 'для',
 'контроля',
 'качества',
 'генеративных']

И в razdel тоже есть токенизация

In [44]:
from razdel import tokenize as razdel_tokenize

In [46]:
list(razdel_tokenize(text))[:10]

[Substring(0, 6, 'Задача'),
 Substring(7, 10, 'NLI'),
 Substring(11, 16, 'важна'),
 Substring(17, 20, 'для'),
 Substring(21, 33, 'компьютерных'),
 Substring(34, 44, 'лингвистов'),
 Substring(44, 45, ','),
 Substring(46, 49, 'ибо'),
 Substring(50, 53, 'она'),
 Substring(54, 63, 'позволяет')]

In [161]:
[token.text for token in list(razdel_tokenize(text))[:10]]

['Задача',
 'NLI',
 'важна',
 'для',
 'компьютерных',
 'лингвистов',
 ',',
 'ибо',
 'она',
 'позволяет']

Работать с регистром тяжело и поэтому можно привести все к нижнему регистру

In [162]:
[token.text.lower() for token in list(razdel_tokenize(text))[:10]]

['задача',
 'nli',
 'важна',
 'для',
 'компьютерных',
 'лингвистов',
 ',',
 'ибо',
 'она',
 'позволяет']

# Нормализация

В последнее время нормализация (т.е. приведение токенов к стандартному виду) используется все реже. Это связано с использованием subword или byte токенизации в топовых моделях (подробнее об этом мы поговорим когда дойдем до нейронных сетей). Однако у них есть свои недостатки и забывать про нормализацию пока не стоит.

Два основных вида нормализации - лемматизация и стемминг. Стемминг уже нигде не используется, но его полезно разобрать, чтобы понимать почему нужно использовать лемматизацию (еще стемминг по непонятной причине часто упоминается как важный скилл в вакансиях)

## Стемминг

Стемминг - это урезание слова до его "основы" (стема), т.е. такой части, которая является общей для всех словоформ в парадигме слова *(Значения слов "слово", "слоформа", "парадигма" приблизительно соответствует тому, которое использует Зализняк вот тут - http://inslav.ru/images/stories/pdf/2002_Zalizniak_RIS_i_statji.pdf (стр. 21-22)) Но это на самом деле не важно)*. 

По крайней мере так в теории. На практике стемминг сводится к отбрасыванию частотных окончаний.

Самый известный стеммер - стеммер Портера (или snowball стеммер). 
Подробнее про стеммер Портера можно почитать вот тут - <https://medium.com/@eigenein/стеммер-портера-для-русского-языка-d41c38b2d340>  
А совсем подробнее вот тут - <http://snowball.tartarus.org/algorithms/russian/stemmer.html>  
Почему он так называется? Так назывался язык программирования, который Портер написал для стеммеров. Язык так называется в созвучие языку SNOBOL. Вот комментарий самого Портера:

`Since it effectively provides a ‘suffix STRIPPER GRAMmar’, I had toyed with the idea of calling it ‘strippergram’, but good sense has prevailed, and so it is ‘Snowball’ named as a tribute to SNOBOL, the excellent string handling language of Messrs Farber, Griswold, Poage and Polonsky from the 1960s.`

Готовые стеммеры для разных языков есть в nltk. Работают они вот так:

In [163]:
from nltk.stem.snowball import SnowballStemmer

In [164]:
stemmer = SnowballStemmer('russian')

In [165]:
[(word, stemmer.stem(word)) for word in word_tokenize(text)][:30]

[('Задача', 'задач'),
 ('NLI', 'NLI'),
 ('важна', 'важн'),
 ('для', 'для'),
 ('компьютерных', 'компьютерн'),
 ('лингвистов', 'лингвист'),
 (',', ','),
 ('ибо', 'иб'),
 ('она', 'он'),
 ('позволяет', 'позволя'),
 ('детально', 'детальн'),
 ('рассмотреть', 'рассмотрет'),
 (',', ','),
 ('какие', 'как'),
 ('языковые', 'языков'),
 ('явления', 'явлен'),
 ('данная', 'дан'),
 ('модель', 'модел'),
 ('понимает', 'понима'),
 ('хорошо', 'хорош'),
 (',', ','),
 ('а', 'а'),
 ('на', 'на'),
 ('каких', 'как'),
 ('–', '–'),
 ('``', '``'),
 ('плывёт', 'плывет'),
 ("''", "'"),
 (';', ';'),
 ('по', 'по')]

Недостатки стемминга достаточно очевидные:  
1) с супплетивными формами или редкими окончаниями слова стемминг работать не умеет  
2) к одной основе могут приводится разные слова  
3) к разным основам могут сводиться формы одного слова  

# Лемматизация

Лемматизация - это замена словоформы слова в парадигме на какую-то заранее выбранную стадартную форму (лемму). 



Например, для разных форм глагола леммой обычно является неопределенная форма (инфинитив), а для существительного форма мужского рода единственного числа. Это позволяет избавиться от недостатков стемминга (будет, был - одна лемма), (пролить, пролом - разные). Однако лемматизация значительно сложнее. 

К счастью есть готовые хорошие лемматизаторы. Для русского основых варианта два: Mystem и Pymorphy.


### Mystem

In [24]:
!pip install pymystem3

  Using cached pymystem3-0.2.0-py3-none-any.whl (10 kB)

[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [26]:
from pymystem3 import Mystem
import os, json
mystem = Mystem(disambiguation=False)


Майстем работает немного лучше и сам токенизирует,
поэтому можно в него засовывать сырой текст.

In [167]:
# mystem.lemmatize функция лемматизации в майстеме
# сам объект mystem нужно заранее инициализировать
mystem.lemmatize(text)[:10]

['задача', ' ', 'NLI', ' ', 'важный', ' ', 'для', ' ', 'компьютерный', ' ']

In [168]:
# Если нужна грамматическая информация или надо сохранить ненормализованный текст,
# есть функция mystem.analyze
words_analized = mystem.analyze(text)

In [169]:
# возвращает она список словарей
# каждый словарь имеет либо одно поле 'text' (когда попался пробел) или text и analysis
# в analysis снова список словарей с вариантами разбора (первый самый вероятный)
# поля в analysis - 'gr' - грамматическая информация, 'lex' - лемма
# analysis - может быть пустым списком
words_analized[:10]

[{'analysis': [{'lex': 'задача', 'wt': 1, 'gr': 'S,жен,неод=им,ед'}],
  'text': 'Задача'},
 {'text': ' '},
 {'analysis': [], 'text': 'NLI'},
 {'text': ' '},
 {'analysis': [{'lex': 'важный', 'wt': 1, 'gr': 'A=ед,кр,жен'}],
  'text': 'важна'},
 {'text': ' '},
 {'analysis': [{'lex': 'для', 'wt': 1, 'gr': 'PR='}], 'text': 'для'},
 {'text': ' '},
 {'analysis': [{'lex': 'компьютерный',
    'wt': 1,
    'gr': 'A=(пр,мн,полн|вин,мн,полн,од|род,мн,полн)'}],
  'text': 'компьютерных'},
 {'text': ' '}]

In [171]:
print('Слово - ', words_analized[0]['text'])
print('Разбор слова - ', words_analized[0]['analysis'][0])
print('Лемма слова - ', words_analized[0]['analysis'][0]['lex'])
print('Грамматическая информация слова - ', words_analized[0]['analysis'][0]['gr'])

Слово -  Задача
Разбор слова -  {'lex': 'задача', 'wt': 1, 'gr': 'S,жен,неод=им,ед'}
Лемма слова -  задача
Грамматическая информация слова -  S,жен,неод=им,ед


In [172]:
#леммы можно достать в одну строчку
[parse['analysis'][0]['lex'] for parse in words_analized if parse.get('analysis')][:10]

['задача',
 'важный',
 'для',
 'компьютерный',
 'лингвист',
 'ибо',
 'она',
 'позволять',
 'детально',
 'рассматривать']

Mystem умеет разбивать текст на предложения, но через питоновский интерфейс это сделать не получится. Нужно скачать mystem отсюда - https://yandex.ru/dev/mystem/

После этого сохранить текст в файл.

In [47]:
f = open('text.txt', 'w')
f.write(text)
f.close()

Из командной строки или из питона запустить майстем на нашем файле

In [48]:
# про параметры почитайте в !mystem -h
!mystem -isc --format json text.txt text_parsed.txt

В целевом файле теперь лежит разобранный текст в jsonlines (json на каждой строчке)

In [49]:
import json
t = [json.loads(line) for line in open('text_parsed.txt')]

In [52]:
t[0][:10]

[{'analysis': [{'lex': 'задача', 'gr': 'S,жен,неод=им,ед'}], 'text': 'Задача'},
 {'text': ' '},
 {'analysis': [], 'text': 'NLI'},
 {'text': ' '},
 {'analysis': [{'lex': 'важный', 'gr': 'A=ед,кр,жен'}], 'text': 'важна'},
 {'text': ' '},
 {'analysis': [{'lex': 'для', 'gr': 'PR='}], 'text': 'для'},
 {'text': ' '},
 {'analysis': [{'lex': 'компьютерный', 'gr': 'A=пр,мн,полн'},
   {'lex': 'компьютерный', 'gr': 'A=вин,мн,полн,од'},
   {'lex': 'компьютерный', 'gr': 'A=род,мн,полн'}],
  'text': 'компьютерных'},
 {'text': ' '}]

Каждый объект в этом списке - параграф. Каждый параграф на предложения можно разбив по тегу '//s'

Ещё так вызывать майстем может понадобиться, если важна скорость.

Недостатки Mystem: это продукт Яндекса с некоторыми ограничениями на использование, больше он не развивается.

Важным достоинством Mystem является то, что он работает не с отдельными словами, а с целым предложением. При определении нужной леммы учитывается контекст, что позволяет во многих случаях разрешать омонимию.

### Pymorphy

Pymorphy - открытый и развивается (но не очень активно, т.к. это все сложно)

Ссылка на репозиторий: https://github.com/kmike/pymorphy2

Попробуйте сразу установить быструю версию (pip install pymorphy2[fast])

У него нет втстроенной токенизации и он расценивает всё как слово. Когда есть несколько вариантов, он выдает их с вероятностостями, которые расчитатны на корпусе со снятой неоднозначностью. 

In [48]:
from pymorphy2.analyzer import Parse, MorphAnalyzer


In [49]:
p = Parse(word='печь', tag='INFN,impf,tran', normal_form='печь', score=0.666666, 
      methods_stack=((dict, 'печь', 2352, 0)))

In [50]:
p._morph = MorphAnalyzer()

In [52]:
# p.inflect(required_grammemes={"gent"})

In [38]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [42]:
# основная функция - pymorphy.parse
words_analized = [morph.parse(token) for token in text.split()]

In [45]:
morph.parse("печь")

In [180]:
# Она похожа на analyze в майстеме только возрващает список объектов Parse
# Первый в списке - самый вероятный разбор (у каждого есть score)
# Информация достается через атрибут (Parse.word - например)
# Грамматическая информация хранится в объекте OpencorporaTag и из него удобно доставать
# части речи или другие категории
print('Первое слово - ', words_analized[0][0].word)
print('Разбор первого слова - ', words_analized[0][0])
print('Лемма первого слова - ', words_analized[0][0].normal_form)
print('Грамматическая информация первого слова - ', words_analized[0][0].tag)
print('Часть речи первого слова - ', words_analized[0][0].tag.POS)
print('Род первого слова - ', words_analized[0][0].tag.gender)
print('Число первого слова - ', words_analized[0][0].tag.number)
print('Падеж первого слова - ', words_analized[0][0].tag.case)

Первое слово -  задача
Разбор первого слова -  Parse(word='задача', tag=OpencorporaTag('NOUN,inan,femn sing,nomn'), normal_form='задача', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'задача', 93, 0),))
Лемма первого слова -  задача
Грамматическая информация первого слова -  NOUN,inan,femn sing,nomn
Часть речи первого слова -  NOUN
Род первого слова -  femn
Число первого слова -  sing
Падеж первого слова -  nomn


## Дополнительная очистка текста

Можно убрать стоп-слова (предлоги, союзы, местоимения, частотные слова). Сам термин стоп-слово происходит из информационного поиска, первый раз его упомянул [Питер Лун](https://en.wikipedia.org/wiki/Hans_Peter_Luhn) в 1959.  
Удаление таких слов позволяло сократить размер индекса и не сильно испортить выдачу или даже улучшить её, поднимая релевантность документам со значимыми словами. Со временем от такой практики, в основном, отказались - память стала дешевой (и повились всякие алгоритмы для сокращения потребления памяти), а для учёта значимости придумали TFIDF (про него на следующем занятии).  

Во многих поисковых движках стоп-слова всё ещё используются. Часто их используют и в практических задачах (классификации, тематическом моделировании). 

In [181]:
from nltk.corpus import stopwords

In [182]:
# стоп-слова есть в nltk
stops = stopwords.words('russian')
print(stops)

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

Список не идеальный и его можно расширять под свои задачи.

In [185]:
words_normalized = [morph.parse(token)[0].normal_form for token in word_tokenize(text)]
[word for word in words_normalized if word not in stops][:10]

['задача',
 'nli',
 'важный',
 'компьютерный',
 'лингвист',
 ',',
 'ибо',
 'позволять',
 'детально',
 'рассмотреть']

## Предобработка для других языков

Предобработка сильно зависит от языка. Полностью универсальнных токенизаторов, лемматизаторов не бывает, а рассказать о предобработке под все существующие языки не реально. Поэтому посмотрим дополнительно только на предобработку на английском языке. 

Nltk и gensim по умолчанию адаптированы под английский язык (а регулярки вообще не привязаны к языку), поэтому разбирать их еще раз не будем.

Библиотека, про которую стоит отдельно рассказать - [**SpaCy**](https://spacy.io/). Это многоцелевая многоязычная библиотека. Если вам понадобится серьезно работать с английским, то лучшим вариантом будет использовать SpaCy. Другие языки там тоже поддерживаются (см. документацию), но не настолько хорошо как английский язык. 

В SpaCy много всего и мы будем возвращаться к ней по ходу курса. Пока посмотрим на интрументы базовой предобработки.

In [57]:
!pip install spacy
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm
!python -m spacy download ru_core_news_sm


[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 2.5 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 2.5 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 2.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 1.3 MB/s eta 0:00:00
  Using cached pymorphy3_dicts_ru-2.4.417150.4580142-py2.py3-none-any.whl (8.4 MB)

[notice] A new release of pip ava

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 7.6 MB/s eta 0:00:0000:0100:01
  Using cached DAWG_Python-0.7.2-py2.py3-none-any.whl (11 kB)
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [56]:
# загружаем пайплайн для английского языка
import spacy

nlp = spacy.load("ru_core_news_sm")


OSError: [E050] Can't find model 'ru_core_news_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [108]:

text = ("One of the most salient features of our culture is that it won't so much bullshit.” "
        "These are the opening words of the short book On Bullshit, written by the philosopher Harry Frankfurt. "
        "Fifteen years after the publication of this surprise bestseller, "
        "the rapid progress of research on artificial intelligence is forcing us to reconsider our conception "
        "of bullshit as a hallmark of human speech, with troubling implications. What do philosophical "
        "reflections on bullshit have to do with algorithms? As it turns out, quite a lot."
        )

In [109]:
doc = nlp(text)

Результат предобаботки очень похож на майстем - тут есть разбиение на предложения, на токены, лемматизация, определение части речи. Тэги тут используются другие, но при желании можно сделать более менее адекватный маппинг (возможно кто-то уже это сделал и нужно только погуглить)

In [110]:
for sent in doc.sents: # достаем предложения
    for token in sent: # достаем токены
        print(token.text, token.lemma_, token.pos_)
    print()

One one NUM
of of ADP
the the DET
most most ADV
salient salient ADJ
features feature NOUN
of of ADP
our our PRON
culture culture NOUN
is be AUX
that that SCONJ
it it PRON
wo wo AUX
n't n't PART
so so ADV
much much ADJ
bullshit bullshit NOUN
. . PUNCT
” " PUNCT

These these DET
are be AUX
the the DET
opening opening NOUN
words word NOUN
of of ADP
the the DET
short short ADJ
book book NOUN
On on ADP
Bullshit Bullshit PROPN
, , PUNCT
written write VERB
by by ADP
the the DET
philosopher philosopher NOUN
Harry Harry PROPN
Frankfurt Frankfurt PROPN
. . PUNCT

Fifteen fifteen NUM
years year NOUN
after after ADP
the the DET
publication publication NOUN
of of ADP
this this DET
surprise surprise NOUN
bestseller bestseller NOUN
, , PUNCT
the the DET
rapid rapid ADJ
progress progress NOUN
of of ADP
research research NOUN
on on ADP
artificial artificial ADJ
intelligence intelligence NOUN
is be AUX
forcing force VERB
us we PRON
to to PART
reconsider reconsider VERB
our our PRON
conception conception

В задачах вроде извлечения ключевых слов может пригодится вытащить из текста только noun phrases (по-русски это вроде называется именные группы существительного)

In [61]:
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])

Noun phrases: ['the most salient features', 'our culture', 'so much bullshit', 'the opening words', 'the short book', 'Bullshit', 'the philosopher', 'Harry Frankfurt', 'the publication', 'this surprise bestseller', 'the rapid progress', 'research', 'artificial intelligence', 'us', 'our conception', 'bullshit', 'a hallmark', 'human speech', 'troubling implications', 'What', 'philosophical reflections', 'bullshit', 'algorithms', 'it']


In [111]:
# загружаем пайплайн для немецкого языка
import spacy

nlp = spacy.load("de_core_news_sm")


In [112]:

text = ("Vor den Stadien habe ich bis jetzt zum Glück noch keine wüsten Szenen gesehen."
        "Vorstandschef Timotheus Höttges habe sich ausgesprochen optimistisch gezeigt, schrieb Analyst Robert Grindle in einer Studie vom Montag. "
        "Während der dortigen Räterepublik war er nach dem Krieg in Künstlergruppen und Ausschüssen aktiv."
        "Welches Ergebnis die Diskussion auf EU-Ebene auch letztlich bringt, wichtig ist, dass die Preisentwicklung für die"
        "Menschen verträglicher gestaltet wird“, so Gusenbauer."
        "Weitere Informationen unter www.schnippenburg.de sowie www.eisenzeithaus.de. Es gibt neue Nachrichten auf noz.de!" 
        "Jetzt die Startseite neu laden."
        "Der Initiative 'Zivilcourage', die sich jahrelang für das Denkmal in Form eines offenen " 
        "Der islamistischen Szene Thüringens wurden nach Angaben des Thüringer Innenministeriums "
        "zuletzt etwa 125 Personen zugerechnet, der salafistischen Szene etwa 75 Personen."
        "Allerdings bestand er die EMV-Prüfung nicht, weil er Radios und DVB-T-Empfänger stört."
        )

In [113]:
doc = nlp(text)

In [114]:
for sent in doc.sents: # достаем предложения
    for token in sent: # достаем токены
        print(token.text, token.lemma_, token.pos_)
    print()

Vor Vor ADP
den der DET
Stadien Stadium NOUN
habe habe AUX
ich ich PRON
bis bis ADP
jetzt jetzt ADV
zum zum ADP
Glück Glück NOUN
noch noch ADV
keine kein DET
wüsten wüst ADJ
Szenen Szene NOUN
gesehen sehen VERB
. . PUNCT

Vorstandschef Vorstandschef NOUN
Timotheus Timotheus PROPN
Höttges Höttges PROPN
habe habe AUX
sich sich PRON
ausgesprochen aussprechen ADV
optimistisch optimistisch ADV
gezeigt zeigen VERB
, , PUNCT
schrieb schreiben VERB
Analyst Analyst NOUN
Robert Robert PROPN
Grindle Grindle PROPN
in in ADP
einer einer DET
Studie Studie NOUN
vom vom ADP
Montag Montag NOUN
. . PUNCT

Während während ADP
der der DET
dortigen dortig ADJ
Räterepublik Räterepublik NOUN
war sein AUX
er ich PRON
nach nach ADP
dem der DET
Krieg Krieg NOUN
in in ADP
Künstlergruppen Künstlergruppen NOUN
und und CCONJ
Ausschüssen Ausschuß NOUN
aktiv aktiv ADV
. . PUNCT

Welches welch DET
Ergebnis Ergebnis NOUN
die der DET
Diskussion Diskussion NOUN
auf auf ADP
EU-Ebene EU-Ebene NOUN
auch auch ADV
letztlich l

In [5]:
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])

Noun phrases: ['den Stadien', 'ich', 'Glück', 'noch keine wüsten Szenen', 'Vorstandschef Timotheus Höttges', 'sich', 'Analyst Robert Grindle', 'einer Studie', 'Montag', 'der dortigen Räterepublik', 'er', 'dem Krieg', 'Künstlergruppen', 'Ausschüssen', 'Welches Ergebnis', 'die Diskussion', 'EU-Ebene', 'die Preisentwicklung', 'dieMenschen', 'Weitere Informationen', 'www.schnippenburg.de', 'www.eisenzeithaus.de', 'neue Nachrichten', 'noz.de', 'Jetzt die Startseite', "Der Initiative 'Zivilcourage", 'die', 'sich', 'das Denkmal', 'Form', 'eines offenen Der islamistischen Szene', 'Thüringens', 'Angaben', 'des Thüringer Innenministeriums', 'zuletzt etwa 125 Personen', 'der salafistischen Szene', 'etwa 75 Personen', 'er', 'die EMV-Prüfung', 'er', 'Radios', 'DVB-T-Empfänger']


Поддержка русского языка в spacy тоже не так давно добавилась, но она не полная

In [55]:
nlp = spacy.load("ru_core_news_sm")

OSError: [E050] Can't find model 'ru_core_news_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [59]:
# возьмем любой текст
text = "ДАННОЕ СООБЩЕНИЕ (МАТЕРИАЛ) СОЗДАНО И (ИЛИ) РАСПРОСТРАНЕНО "\
       "ИНОСТРАННЫМ СРЕДСТВОМ МАССОВОЙ ИНФОРМАЦИИ, ВЫПОЛНЯЮЩИМ "\
       "ФУНКЦИИ ИНОСТРАННОГО АГЕНТА, И (ИЛИ) РОССИЙСКИМ ЮРИДИЧЕСКИМ ЛИЦОМ, "\
       "ВЫПОЛНЯЮЩИМ ФУНКЦИИ ИНОСТРАННОГО АГЕНТА"

In [60]:
doc = nlp(text)

for sent in doc.sents: # достаем предложения
    for token in sent: # достаем токены
        print(token.text, token.lemma_, token.pos_)

ДАННОЕ ДАННОЕ PROPN
СООБЩЕНИЕ СООБЩЕНИЕ PROPN
( ( PUNCT
МАТЕРИАЛ МАТЕРИАЛ PROPN
) ) PUNCT
СОЗДАНО СОЗДАНО PROPN
И И PROPN
( ( PUNCT
ИЛИ ИЛИ PROPN
) ) PUNCT
РАСПРОСТРАНЕНО РАСПРОСТРАНЕНО PROPN
ИНОСТРАННЫМ ИНОСТРАННЫМ PROPN
СРЕДСТВОМ СРЕДСТВОМ PROPN
МАССОВОЙ МАССОВОЙ VERB
ИНФОРМАЦИИ ИНФОРМАЦИИ PROPN
, , PUNCT
ВЫПОЛНЯЮЩИМ ВЫПОЛНЯЮЩИМ PROPN
ФУНКЦИИ ФУНКЦИИ NOUN
ИНОСТРАННОГО иностранного NOUN
АГЕНТА агента NOUN
, , PUNCT
И И PROPN
( ( PUNCT
ИЛИ ИЛИ PROPN
) ) PUNCT
РОССИЙСКИМ РОССИЙСКИМ VERB
ЮРИДИЧЕСКИМ ЮРИДИЧЕСКИМ PROPN
ЛИЦОМ ЛИЦОМ PROPN
, , PUNCT
ВЫПОЛНЯЮЩИМ ВЫПОЛНЯЮЩИМ PROPN
ФУНКЦИИ ФУНКЦИИ NOUN
ИНОСТРАННОГО иностранного NOUN
АГЕНТА агента NOUN


In [61]:
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])

Noun phrases: ['ДАННОЕ СООБЩЕНИЕ', 'МАТЕРИАЛ', 'ИНОСТРАННЫМ СРЕДСТВОМ', 'ВЫПОЛНЯЮЩИМ ФУНКЦИИ ИНОСТРАННОГО АГЕНТА', 'И', 'ИЛИ', 'ЮРИДИЧЕСКИМ', 'ЛИЦОМ', 'ВЫПОЛНЯЮЩИМ ФУНКЦИИ ИНОСТРАННОГО АГЕНТА']


# Удаление дубликатов

В многих практических задачах требуется искать совпадающие тексты. Например, для определения плагиата, или для того, чтобы отобрать уникальные тексты для обучения. Сама по себе эта задача - тривиальная. Нужно просто сравнить тексты между собой. Даже если нужно найти не просто точно совпадающие тексты (дупликаты), а еще и похожие тексты. В этом случае можно заменить прямое сравнение какой-то метрикой (например, мерой Жаккара между множествами нграммов).

Но и то и другое становится проблемой, когда количество текстов очень большое. Чтобы найти дубликаты нужно сравнить все тексты со всеми и это слишком много вычислений даже для самых простых методов.

Для такой задачи стандартно применяется класс алгоритмов, которые называются Local Sensitive Hashing. Давайте попробуем разобраться как это работает.

Тут три важных компоненты: шинглы, minhash и LSH (это все в целом называется LSH но и последний шаг тоже так называется, что немного запутанно)

1) Шинглы - это просто куски текстов/документов какой-то длины. Мы бы скорее назвали это символьными нграммами.
2) Minhash - это алгоритм, который позволяет рассчитать приблизительное расстояние Жаккара между множествами шинглов двух документов (https://en.wikipedia.org/wiki/MinHash). Это не единственный алгоритм, еще есть например SimHash (https://en.wikipedia.org/wiki/SimHash)
3) LSH шаг это еще одна оптимизация для нахождения кандидатов в дубликаты

Пройдемся по каждому из шагов и напишем простую реализацию на питоне

![](https://media.springernature.com/lw1200/springer-static/image/art%3A10.1007%2Fs10660-021-09472-1/MediaObjects/10660_2021_9472_Fig2_HTML.png)



Сгенерировать шинглы очень просто. Единственный важный момент что в итоге нам нужны только уникальные шинглы (то есть множества а не списки)

Длина шингла это параметр который можно настраивать

In [81]:
def get_shingles(text, k=5):
    """генерирует список шинглов из строки"""
    shingles = set()
    for i in range(len(text) - k + 1):
        shingle = text[i:i + k]
        shingles.add(shingle)
    return shingles

In [200]:
get_shingles("пример строки для теста", 8)

{' для тес',
 ' строки ',
 'для тест',
 'ер строк',
 'и для те',
 'имер стр',
 'ки для т',
 'ля теста',
 'мер стро',
 'оки для ',
 'пример с',
 'р строки',
 'ример ст',
 'роки для',
 'строки д',
 'троки дл'}

MinHash самая сложная часть этого алгоритма. 

Можете посмотреть одно или несколько из этих видео, чтобы разобрать поглубжe:  
https://www.youtube.com/watch?v=e_SBq3s20M8  
https://www.youtube.com/watch?v=96WOGPUgMfw  
https://www.youtube.com/watch?v=R12splIFMOs  
https://www.youtube.com/watch?v=bQAYY8INBxg 



Во-первых, нужно заменить шинглы на числа с помощью хеширования. Хэширование это маппинг элементов (например строк) в позиции в какой-то таблице. Позиция в таблице и есть хэш, обычно это просто число. Маппинг происходит с помощью какой-то функции и поэтому с помощью хеширования удобно проверять наличие - достаточно расчитать хэш объекта и посмотреть есть ли уже такой объект в таблице, сравнивать со всеми существующими объектами не нужно

Словари и множества в питоне работают на хеш таблицах и поэтому проверить есть ли какой-то элемент в множестве гораздо быстрее чем в списке 

In [96]:
d = {str(i) for i in range(1000000)}
l = [str(i) for i in range(1000000)]

In [97]:
%%time
"8732323" in d

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 8.11 µs


False

In [100]:
%%time
"873232312" in l

CPU times: user 18.3 ms, sys: 354 µs, total: 18.7 ms
Wall time: 19.7 ms


False

Функций для хеширования очень много и это своя отдельная тема. Для минхэша нам пондобится генерировать много хэш функций, это можно сделать вот так - мы используем один алгоритм хэширования, но к строкам добавляем какой-то индекс, что меняет результат

In [207]:
import hashlib
from collections import defaultdict


def hash_string(s):
    """хеширует строку и возвращает число"""
    return int(hashlib.md5(s.encode('utf8')).hexdigest(), 16) 

def generate_hash_functions(k):
    """генерирует k хеш-функций добавляя индекс к строке"""

    functions = []
    for i in range(k):
        functions.append(lambda x, i=i: hash_string(x + str(i)))
    return functions

In [161]:
# import sympy

# sympy.isprime(5)

# list(sympy.primerange(100000, 1100000))

In [163]:
%%time
int(hashlib.md5(texts[220].encode('utf-8')).hexdigest(), 16) % 105337

CPU times: user 52 µs, sys: 2 µs, total: 54 µs
Wall time: 58.2 µs


17236

In [165]:
# если вызвать эту функцию на одной строке несколько раз то результат будет одинаковый
hash_string("пример")

9238

Сделаем две функции чтобы генерировать разные результаты для одной строки 

In [166]:
functions = generate_hash_functions(2)

In [167]:
functions[0]("пример")

69815

In [168]:
functions[1]("пример")

49600

Теперь самая сложная часть. Вместо хэшей нужно сгенерировать их сигнатуры, которые будут приближать меру Жаккара между множествами шинглов

Этот алгоритм обычно объясняют вот такой схемой. 

![](https://miro.medium.com/v2/resize:fit:913/1*4HbYE4-7DRfri7cfN0tN0A.png)

В центре матрица шинглы на документы (input matrix). Такие же матрицы мы будем еще раз много строить для представления текстов мешком слов. Это разреженная матрица где 1 показывает это слово есть в документе, а 0 что нет. Каждый документ представляется вектором, который равен размеру всего словаря (то есть количеству уникальных слов/токенов/шинглов во всех документах). 

Такие вектора можно использовать для расчета метрики Жаккара (или для косиносной близости), но это все еще слишком дорого, потому что нужно сравнивать все со всем. Метрика Жаккара между двумя такими векторами равна количеству совпадений единиц на одной позиции, поделить на количество ненулевых позиций в обоих векторах. Если в конкретной позиции у обоих векторов стоят 1, значит слово/шингл есть в обоих документах. А если 1 стоит только в одном векторе, а в другом 0 - то значит пересечения между ними нет. Нули тут никак не учитываются. Это то же самое, что рассчитать пересечение множеств и поделить на объединение, только в векторном формате.

MinHash это такое преобразование этой матрицы шинглы на документы, которое приближает меру жаккара. Преобразование заключается в том, что генерируются перестановки для этой матрицы и в каждой перестановке для каждого документа находится первая ненулевая позиция и ее индекс записывается в новую матрицу (сигнатуру). Перестановки тут обозначены цветными векторами. Если вы посмотрите на голубой вектор, то первой теперь является строчка, которая была 6-ой в изначальной матрице. В ней у первого документа уже стоит 1, поэтому для первого документа значение сигнатуры - 1 (матрица справа). Для второго документа стоит 0, поэтому нужно смотреть следующий вектор - 4-ый в изначальной матрице. В нем уже стоит 1 во втором документе, поэтому значение сигнатуры будет 2. У третьего документа 1 потому что в первом векторе уже стоит 1, а в 4 такая же ситуация как и во втором. (Проверьте что вы понимаете как получились желтая и коричневая сигнатура)

Теперь сигнатуры документов можно использовать для расчета Жаккара так же как и обычные вектора. Если позиции сходятся, то документы схожи. Это гораздо эффективнее потому что сигнатуры гораздо меньше изначальных векторов и большую часть вычисление делать теперь не нужно.

Каждая сигнатура может не точно передавать общую близость изначальных векторов, но если сделать много таких перестановок, то схожесть по сигнатурам будет очень близка схожести по изначальным векторам.
Такой подход приближает метрику Жаккара, потому что если в обоих документах много совпадений шинглов (1 и 1 в обоих на одной позиции), то они часто будут генерировать один и тот же индекс для сигнатуры. Чем больше несовпадений шинглов в документах (0 в одном и 1 в другом), тем чаще в сигнатуре будут разные индексы и соответственно метрика будет низкая. Получается даже так, что вероятность при случайной перестановке получить 1 и 1 в обоих векторах равна метрике Жаккара между изначальными документами! (количество случаев где оба значения 1, поделить на количество ненулевых позиции в обоих векторах). Это только вероятность и каждая отдельная перестановка может давать другой результат, но повторяя их много раз, в среднем мы получим исходное расстояние. 




В коде это выглядит немного по-другому, но принцип точно такой же! Просто перестановки создавать сложно и делать полную матрицу тоже не нужно. Можно просто вместо перестановок делать разные хеш функции (хеш функция это просто маппинг в число, то есть в итоге мы получаем позицию). Если применить хеш функцию ко всем шинглам в документе и взять минимальное значение, то это то же самое что сделать перестановку и взять первое ненулевое значение вектора выше!

Поэтому код очень простой.

In [169]:
def compute_minhash_signature(shingles, hash_funcs):
    """вычисляет minhash-сигнатуру для списка шинглов"""
    signature = []
    for hash_func in hash_funcs:
        min_hash = min(hash_func(shingle) for shingle in shingles)
        signature.append(min_hash)
    return signature

In [170]:
# числа тут гораздо больше чем в примере
# просто неудобно показывать такие большие индексы для примера
# но суть от этого не меняется - это просто числа индексы
shingles = get_shingles("пример строки для теста", 3)
hash_funcs = generate_hash_functions(5)
compute_minhash_signature(shingles, hash_funcs)

[4888, 8891, 9010, 8604, 401]

Последний элемент LSH просто разрезает сигнатуры на кусочки и группирует документы по совпадению этих кусочков. Сгруппированные документы уже являются кандидатами для расчета нормальной близости, потому что они скорее всего будут дубликатами.

In [183]:
def lsh(signatures, bands):
    """Разрезает сигнатуры на куски (bands), и группирует индексы сигнатур по совпадению кусков"""
    buckets = defaultdict(list)
    band_length = len(signatures[0]) // bands
    
    for idx, sig in tqdm(enumerate(signatures)):
        for b in range(0, bands, band_length):
            start = b
            end = start + band_length
            band = tuple(sig[start:end])
            buckets[band].append(idx)
    return buckets

In [172]:
signature_1

[47768649540959415773458792523333048870,
 7684655364199927409860563777974756678,
 15083700868336705187602456618714281847,
 23751230482080805943145997883796637502,
 6444901249487694733329551606909628849,
 3440871066400609071968054431171020532]

In [173]:
hash_funcs = generate_hash_functions(6)

shingles_1 = get_shingles("пример строки для теста", 3)
signature_1 = compute_minhash_signature(shingles_1, hash_funcs)

shingles_2 = get_shingles("совершенно другой текст", 3)
signature_2 = compute_minhash_signature(shingles_2, hash_funcs)

shingles_3 = get_shingles("пример похожей строки для теста", 3)
signature_3 = compute_minhash_signature(shingles_3, hash_funcs)

lsh([signature_1, signature_2, signature_3], 2)

# 0, 2 сгруппировались по совпадению кусочков сигнатур длинной 3

defaultdict(list,
            {(4888, 8891, 9010): [0],
             (9958, 824, 727): [1],
             (3244, 8891, 9010): [2]})

In [205]:
def find_similar_strings(strings_list, k=5, num_hashes=100, bands=20):
    """Finds similar strings using MinHash and LSH."""
    hash_funcs = generate_hash_functions(num_hashes)
    signatures = []
    shingles_list = []

    # каждый текст обрабатывается отдельно
    # находятся шинглы и рассчитываются сигнатуры 
    for string in tqdm(strings_list):
        shingles = get_shingles(string, k)
        shingles_list.append(shingles)
        signature = compute_minhash_signature(shingles, hash_funcs)
        signatures.append(signature)

    # вычисляются кандидаты по кускам сигнатур
    buckets = lsh(signatures, bands)
    candidates = set()
    for bucket in buckets.values():
        if len(bucket) > 1:
            for i in bucket:
                for j in bucket:
                    if i < j:
                        candidates.add((i, j))


    return candidates



In [320]:
# Example usage
strings_list = [
    "This is a sample string for testing",
    "This is a sample string for testing",
    "This is a simple string for testing",
    "Completely different string here",
    "Another different string for testing purposes",
    "This is a sample string for testing and more"
]

candidates = find_similar_strings(strings_list)

  0%|          | 0/6 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [186]:
candidates

{(0, 1), (0, 5), (1, 5)}

Чтобы быть уверенным в метрике можно рассчитать полное расстояние между кандидатами

In [321]:
def get_jaccard(x, y):
    x_sh = get_shingles(x, 10)
    y_sh = get_shingles(y, 10)

    return len(x_sh & y_sh) / len(x_sh | y_sh)

In [326]:
(get_jaccard(strings_list[0], strings_list[1]), 
get_jaccard(strings_list[0], strings_list[5]), 
get_jaccard(strings_list[1], strings_list[5]))

(1.0, 0.7428571428571429, 0.7428571428571429)

# Загрузка текста из html

Вообще все сайты разные и доставать текст из каждого нужно по-своему, но многие вещи и паттерны повторяются и поэтому можно извлекать большинство текстов уже готовыми скриптами и библиотеками (например есть trafilatura)


In [11]:
# !pip install trafilatura lxml_html_clean

In [78]:
import trafilatura
downloaded = trafilatura.fetch_url('https://github.blog/2019-03-29-leader-spotlight-erin-spiceland/')
trafilatura.extract(downloaded)

'Leader spotlight: Erin Spiceland\nWe’re spending Women’s History Month with women leaders who are making history every day in the tech community. Read more about Erin Spiceland: Software Engineer at SpaceX.\nEvery March we recognize the women who have shaped history—and now, we’re taking a look forward. From driving software development in large companies to maintaining thriving open source communities, we’re spending Women’s History Month with women leaders who are making history every day in the tech community. Erin Spiceland is a Software Engineer for SpaceX. Born and raised in rural south Georgia, she is a Choctaw and Chickasaw mother of two now living in downtown Los Angeles. Erin didn’t finish college—she’s a predominantly self-taught software engineer. In her spare time, she makes handmade Native American beadwork and regalia and attends powwows.\nHow would you summarize your career (so far) in a single sentence?\nMy career has been a winding road through periods of stimulation

# Определение языка (fastext language classifier)

Как и в любой задаче, в определении языка есть свои особенности и сложности, в которые можно закопаться очень глубоко. Но глобально - это стандартная задача классификации и мы будем разбирать ее еще много раз. Единственное, что хотелось сказать - это fasttext language classifier. Это очень распространенный предобученный классификатор языков, который часто используется в статьях (в статье про FineWeb он тоже упоминается).
Больше информации про этот классификатор - https://fasttext.cc/docs/en/language-identification.html (там же есть ссылки на скачивание модели)

In [21]:
# !pip install fasttext

In [20]:
# !wget https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.ftz

In [76]:
import fasttext

In [77]:
model = fasttext.load_model('lid.176.ftz')

In [19]:
# классификатор выдает самый вероятный язык и саму вероятность
model.predict('Язык предложения')

(('__label__ru',), array([0.9825241]))